$
\begin{aligned}
\texttt{JBOD\_MOUNT}           &= 000000 \\
\texttt{JBOD\_UNMOUNT}         &= 000001 \\
\texttt{JBOD\_SEEK\_TO\_DISK}  &= 000010 \\
\texttt{JBOD\_SEEK\_TO\_BLOCK} &= 000011 \\
\texttt{JBOD\_READ\_BLOCK}     &= 000100 \\
\texttt{JBOD\_WRITE\_BLOCK}    &= 000101 \\
\end  {aligned}
$

```c
int jbod_operation (uint32_t op, uint8_t *block)
//                  ^^^^^^^^^^^                  32-bit unsigned int             INSTRUCTION
//                               ^^^^^^^^^^^^^^   8-bit unsigned int, pointer to buffer

if success
  return 0
else
  return -1
```

```c
int jbod_operation (JBOD_MOUNT, NULL)

int jbod_operation (JBOD_UNMOUNT, NULL)

int jbod_operation (JBOD_SEEK_TO_DISK, NULL) {
  {CurrentDiskID = DiskID, CurrentBlockID = 0}
}

int jbod_operation (JBOD_SEEK_TO_BLOCK, NULL) {
  {CurrentDiskID, CurrentBlockID = BlockID}
}

int jbod_operation (JBOD_READ_BLOCK, buffer) {
  buffer = malloc(sizeof(uint8_t) * 256); // 256 bytes
  memcpy();

  {CurrentDiskID, CurrentBlockID = CurrentBlockID + 1 block}
}

int jbod_operation (JBOD_WRITE_BLOCK, buffer) {
  buffer = malloc(sizeof(uint8_t) * 256); // 256 bytes
  memcpy();

  {CurrentDiskID, CurrentBlockID = CurrentBlockID + 1 block}
}
```

In [6]:
DISKS                       =  16
BLCKS_PER_DISK              = 256
BYTES_PER_BLOCK = BLCK_SIZE = 256
DISK_SIZE                   = BLCK_SIZE * BLCKS_PER_DISK
JBOD_SIZE                   = DISK_SIZE * DISKS

print(f'disk size  : {DISK_SIZE:,} B')
print(f'jbod size  : {JBOD_SIZE:,} B')

disk size  : 65,536 B
jbod size  : 1,048,576 B


In [7]:
start_addr = 224
read_len   = 2**10

assert(0 <= start_addr <= 2**20)
assert(0 <= read_len   <= 2**10)

print(f'start_addr : {start_addr:>10,}')
print(f'read_len   : {read_len  :>10,}')

start_addr :        224
read_len   :      1,024


In [8]:
disk_id = start_addr // DISK_SIZE
blck_id = start_addr  % DISK_SIZE // BLCK_SIZE
ofst    = start_addr  % DISK_SIZE  % BLCK_SIZE

print(f'disk : {disk_id}')
print(f'blck : {blck_id}')
print(f'ofst : {ofst}')
orig_addr = disk_id * DISK_SIZE + blck_id * BLCK_SIZE + ofst
print(f'{disk_id} disks x DISK_SIZE + {blck_id} blcks x BLCK_SIZE + {ofst} = {orig_addr}')

disk : 0
blck : 0
ofst : 224
0 disks x DISK_SIZE + 0 blcks x BLCK_SIZE + 224 = 224


In [43]:
             # requirements
sadr = 102380 - 236   ; assert(0 <= sadr < 2**20)
rdln = 102 ; assert(0 <= rdln <= 2**10)

disk = sadr // DISK_SIZE
blck = sadr  % DISK_SIZE // BLCK_SIZE
ofst = sadr  % DISK_SIZE  % BLCK_SIZE

# print(f'sadr : {sadr:>10,}')
# print(f'rdln : {rdln:>10,}')
# print(f'ofst : {ofst:>10,}')
# print()
print(f'  { disk :>5} { blck :>5} { ofst :>5} { sadr :>10} { rdln :>5}')
print(f"  {'disk':>5} {'blck':>5} {'ofst':>5} {'sadr':>10} {'rdln':>5} {'numb':>5}")
print()

i = 0
while (rdln > 0):
  numb = BLCK_SIZE - ofst if rdln >= (BLCK_SIZE - ofst) else rdln #- ofst


  print(f'{i} {disk:>5} {blck:>5} {ofst:>5} {sadr:>10} {rdln:>5} {numb:>5}')

  rdln -= numb
  sadr += numb

  disk = sadr // DISK_SIZE
  blck = sadr  % DISK_SIZE // BLCK_SIZE
  ofst = sadr  % DISK_SIZE  % BLCK_SIZE

  i += 1

print(f'{i} {disk:>5} {blck:>5} {ofst:>5} {sadr:>10} {rdln:>5} {"0":>5}')

      1   143     0     102144   102
   disk  blck  ofst       sadr  rdln  numb

0     1   143     0     102144   102   102
1     1   143   102     102246     0     0
